# Welcome to the Lab 🥼🧪

In [ ]:
import os
import sys
import json

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido numpy
    !git clone https://github.com/ParclLabs/parcllabs-examples.git
    sys.path.append('/content/parcllabs-examples/python/')
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')
    cur_dir = os.getcwd()
    chart_dir = os.path.join(cur_dir, '..', '..')
    sys.path.append(chart_dir)

In [ ]:
import os
import parcllabs
from parcllabs import ParclLabsClient

api_key = os.getenv('PARCL_LABS_API_KEY')
print(f"Parcl Labs Version: {parcllabs.__version__}")



# Initialize the Parcl Labs client
client = ParclLabsClient(api_key=api_key, limit=100)

# Get top 100 CBSAs by population
markets = client.search_markets.retrieve(
    location_type='CBSA',
    sort_order='DESC',
    sort_by='TOTAL_POPULATION',
    as_dataframe=True
)

housing_events = client.market_metrics_housing_event_prices.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2022-05-01'
)

print(housing_events)

START_DATE = '2023-06-01'
data = client.portfolio_metrics_sf_housing_event_counts.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    portfolio_size='PORTFOLIO_1000_PLUS'
)
print(data)
# get supply
inventory = client.for_sale_market_metrics_for_sale_inventory.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date=START_DATE,
)

print(inventory)

# get demand
demand = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2023-05-01',
)

print(demand)



In [ ]:
import parcllabs
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from parcllabs import ParclLabsClient

api_key = os.getenv('PARCL_LABS_API_KEY')
print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# import technical charts
from charting.utils import create_labs_logo_dict
from charting.pricefeed_utils import TimeSeriesAnalysis
from charting.name_formats import format_names, create_ticker, format_cs_10_names
from charting.technical_charts import technical_pricefeed_charts, calculate_stats


In [ ]:
# Initialize the Parcl Labs client
client = ParclLabsClient(api_key)

In [ ]:
# Get top 100 CBSAs by population
markets = client.search_markets.retrieve(
    as_dataframe=True,
    location_type='CBSA',
    sort_order='DESC',
    sort_by='TOTAL_POPULATION',
    params={
        'limit': 50
    }
)
markets['clean_name'] = markets['name'].apply(format_cs_10_names)
markets

In [ ]:
# lets retrieve data back to 2011 for these price feeds
START_DATE = '2023-06-01'
price_changes = client.for_sale_market_metrics_for_sale_inventory_price_changes.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
    auto_paginate=True, # auto paginate to get all the data - WARNING: ~6k credits can be used in one parcl price feed. Change the START_DATE to a more recent date to reduce the number of credits used
)

In [ ]:
# get supply and demand
inventory = client.for_sale_market_metrics_for_sale_inventory.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date=START_DATE,
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
)

In [ ]:
# get supply and demand
demand = client.market_metrics_housing_event_counts.retrieve_many(
    parcl_ids=markets['parcl_id'].tolist(),
    start_date='2023-05-01',
    as_dataframe=True,
    params={'limit': 1000},  # expand the limit to 1000, these are daily series
)

In [ ]:
chart = inventory.merge(price_changes[['date', 'parcl_id', 'count_price_drop']], on=['date', 'parcl_id'], how='inner')

In [ ]:
chart['pct_price_drop'] = chart['count_price_drop'] / chart['for_sale_inventory']
chart

In [ ]:
chart.sort_values('pct_price_drop', ascending=False)

In [ ]:
markets.loc[markets['parcl_id']==2900417]

In [ ]:
data = chart.loc[chart['parcl_id']==2900417]

In [ ]:
import plotly.graph_objects as go
import pandas as pd


# config logo
labs_logo_dict = create_labs_logo_dict(
    src='labs',
    y=1.04,
    x=0.5,
    xanchor='center',
    yanchor='top',
    sizex=0.15,
    sizey=0.15,
)

def demand_vs_pf_chart(
    data,
    line_chart_series: str = 'for_sale_inventory',
    bar_chart_series: str = 'pct_price_drop',
    save_path: str = None,
    title: str = None,
    line_chart_title: str = 'Percentage Price Drop',
    bar_chart_title: str = 'For Sale Inventory'
):
    HEIGHT = 900
    WIDTH = 1600
    
    fig = go.Figure()

    # Add bar trace for For Sale Inventory
    fig.add_trace(go.Bar(
        x=data['date'],
        y=data[bar_chart_series],
        marker=dict(color='#636EFA', opacity=0.7),
        name=bar_chart_title,
        yaxis='y2'
    ))

    # Add primary y-axis trace for Percentage Price Drop
    fig.add_trace(go.Scatter(
        x=data['date'],
        y=data[line_chart_series] * 100,  # Convert to percentage
        mode='lines+markers',
        line=dict(width=4, color='#00CC96'),  # Solid green color for price series
        marker=dict(size=6),
        name=line_chart_title
    ))

    # Reorder traces so the line is in front
    fig.data = fig.data[::-1]

    # Add logo image
    fig.add_layout_image(
        labs_logo_dict
    )
    
    fig.update_layout(
        margin=dict(l=20, r=20, t=110, b=20),
        height=HEIGHT,
        width=WIDTH,
        title={
            'text': title,
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=28, color='#FFFFFF'),
        },
        plot_bgcolor='#222222',
        paper_bgcolor='#222222',
        font=dict(color='#FFFFFF'),
        xaxis=dict(
            title_text='',
            showgrid=True,
            gridwidth=0.5,
            gridcolor='rgba(255, 255, 255, 0.1)',
            tickangle=-45,
            tickfont=dict(size=14),
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1
        ),
        yaxis=dict(
            title_text=line_chart_title,
            showgrid=True,
            gridwidth=0.5,
            gridcolor='rgba(255, 255, 255, 0.2)',
            tickfont=dict(size=14),
            ticksuffix='%',
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1
        ),
        yaxis2=dict(
            title_text=bar_chart_title,
            showgrid=True,
            gridwidth=0.5,
            gridcolor='rgba(255, 255, 255, 0.2)',
            tickfont=dict(size=14),
            zeroline=False,
            linecolor='rgba(255, 255, 255, 0.7)',
            linewidth=1,
            overlaying='y',
            side='right',
            ticksuffix=' units'
        ),
        hovermode='x unified',
        hoverlabel=dict(
            bgcolor='#333333',
            font_size=14,
            font_family="Rockwell"
        ),
        legend=dict(
            x=0.02,  # Position legend in the top right corner
            y=0.98,
            xanchor='left',
            yanchor='top',
            font=dict(size=14, color='#FFFFFF'),
            bgcolor='rgba(0, 0, 0, 0.5)'
        ),
        barmode='stack'  # Stack the bars
    )

    if save_path:
        fig.write_image(save_path, width=WIDTH, height=HEIGHT)
    fig.show()


In [ ]:
demand_vs_pf_chart(
    data,
    line_chart_series = 'pct_price_drop',
    bar_chart_series= 'for_sale_inventory',
    save_path = None,
    title = 'Tampa, FL Supply vs. Pct of Supply w/Price Drops',
    line_chart_title = '% of Supply w/Price Drop',
    bar_chart_title = 'For Sale Inventory'
)

In [ ]:
demand.loc[demand['parcl_id']==2900417]